In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dir = '/content/drive/MyDrive/data'
! ls '/content/drive/MyDrive/data'

testing  training  validation


In [ ]:
bahan_dir = os.path.join(base_dir, 'testing')
train_dir = os.path.join(base_dir, 'traning')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
train_dir = os.path.join(base_dir, 'training')
train_dogs_dir = os.path.join(train_dir, 'dogs')
train_wolves_dir = os.path.join(train_dir, 'wolves')

print("Jumlah Data Train Tiap Kelas")
print('Jumlah gambar Dogs :', len(os.listdir(train_dogs_dir)))
print('Jumlah gambar Wolves :', len(os.listdir(train_wolves_dir)))


Jumlah Data Train Tiap Kelas
Jumlah gambar Dogs : 1000
Jumlah gambar Wolves : 1000


In [ ]:
import os
import random
from shutil import copyfile
from google.colab import drive

# Fungsi untuk membagi dataset menjadi train & validation
def train_val_split(source, train, val, train_ratio):
    # Cek apakah folder source ada
    if not os.path.exists(source):
        print(f"Error: Folder '{source}' tidak ditemukan!")
        return

    # Ambil daftar semua file dalam folder source
    all_files = os.listdir(source)
    total_size = len(all_files)

    # Jika folder kosong, hentikan proses
    if total_size == 0:
        print(f"Folder '{source}' kosong, tidak ada data untuk diproses!")
        return

    # Buat folder train & val jika belum ada
    os.makedirs(train, exist_ok=True)
    os.makedirs(val, exist_ok=True)

    # Hitung jumlah data train & validation
    train_size = int(train_ratio * total_size)

    # Acak urutan file
    randomized = random.sample(all_files, total_size)

    # Pisahkan file untuk train & validation
    train_files = randomized[:train_size]
    val_files = randomized[train_size:]

    # Copy file ke folder train
    for i in train_files:
        i_file = os.path.join(source, i)
        destination = os.path.join(train, i)
        copyfile(i_file, destination)

    # Copy file ke folder validation
    for i in val_files:
        i_file = os.path.join(source, i)
        destination = os.path.join(val, i)
        copyfile(i_file, destination)

    print(f"Data dari '{source}' telah dibagi menjadi:")
    print(f"   Train: {train_size} file → {train}")
    print(f"   Validation: {len(val_files)} file → {val}\n")

# Struktur dataset
dogs_dir = os.path.join(base_path, "testing/dogs")
wolves_dir = os.path.join(base_path, "testing/wolves")

# Folder tujuan setelah dipisah
train_dogs = os.path.join(base_path, "training/dogs")
validation_dogs = os.path.join(base_path, "validation/dogs")

train_wolves = os.path.join(base_path, "training/wolves")
validation_wolves = os.path.join(base_path, "validation/wolves")

# Tentukan rasio train-validation
train_ratio = 0.9

# Jalankan proses split
train_val_split(dogs_dir, train_dogs, validation_dogs, train_ratio)
train_val_split(wolves_dir, train_wolves, validation_wolves, train_ratio)

# Tampilkan jumlah file setelah split
print("Jumlah semua dogs   :", len(os.listdir(dogs_dir)))
print("Jumlah train dogs   :", len(os.listdir(train_dogs)))
print("Jumlah val dogs     :", len(os.listdir(validation_dogs)))

print("Jumlah semua wolves :", len(os.listdir(wolves_dir)))
print("Jumlah train wolves :", len(os.listdir(train_wolves)))
print("Jumlah val wolves   :", len(os.listdir(validation_wolves)))


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation untuk training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    horizontal_flip=True,
    shear_range=0.3,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
)

# Data augmentation untuk validation (hanya rescale)
val_datagen = ImageDataGenerator(
    rescale=1./255
)

# Path dataset setelah split
base_path = "/content/drive/MyDrive/data"

train_dir = os.path.join(base_path, "training")
val_dir = os.path.join(base_path, "validation")

# Load dataset menggunakan ImageDataGenerator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Ukuran gambar yang diubah sesuai model
    batch_size=32,
    class_mode='binary'  # Karena hanya ada dua kelas: dogs dan wolves
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 2001 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print('\nAkurasi mencapai 99%')
      self.model.stpo_training = True

callbacks = myCallback()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(150, 150, 3)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.3,seed=112),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.5,seed=112),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 18496)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 200)                 │       3,699,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 500)                 │         100,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │           1,002 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,824,486 (14.59 MB)

 Trainable params: 3,824,486 (14.59 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='binary_crossentropy',  # Ubah ke binary_crossentropy untuk 2 kelas
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),  # Gunakan jumlah batch per epoch dari train_generator
    epochs=25,
    validation_data=val_generator,
    validation_steps=len(val_generator),  # Gunakan jumlah batch per epoch dari val_generator
    verbose=1,
    callbacks=[callbacks]
)


Epoch 1/25
 88/201 ━━━━━━━━━━━━━━━━━━━━ 2:54 2s/step - accuracy: 0.4735 - loss: 0.6938

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7ed93c078630>
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 248, in _finite_generator
    yield self._standardize_ba